In [ ]:
import pandas as pd
import numpy as np

def SMA(series, window):
    return series.rolling(window=window).mean()

def EMA(series,window):
    alpha=2/(window+1)
    ema=[]
    for i in range(len(series)):
        if i==0:
          ema.append(series.iloc[0])
        else:
          value=alpha*series.iloc[i]+(1-alpha)*ema[i-1]
          ema.append(value)
    return pd.Series(ema,index=series.index)

def MACD(series):
    ema12 = EMA(series, 12)
    ema26 = EMA(series, 26)
    macd_line = ema12 - ema26
    signal_line = EMA(macd_line, 9)
    histogram = macd_line - signal_line

    return macd_line, signal_line, histogram

def RSI(series, window=14):
    delta = series.diff()

    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def Stochastic(high, low, close, window=14):
    lowest_low = low.rolling(window).min()
    highest_high = high.rolling(window).max()

    percent_k = 100 * (close - lowest_low) / (highest_high - lowest_low)
    percent_d = percent_k.rolling(3).mean()

    return percent_k, percent_d

def Bollinger_Bands(series, window=20):
    sma = SMA(series, window)
    std = series.rolling(window).std()

    upper_band = sma + (2 * std)
    lower_band = sma - (2 * std)

    return sma, upper_band, lower_band


import yfinance as yf

reliance = yf.download("RELIANCE.NS", period="2y")
# Moving Averages
reliance["SMA_20"] = SMA(reliance["Close"], 20)
reliance["EMA_20"] = EMA(reliance["Close"], 20)

# MACD
reliance["MACD"], reliance["Signal"], reliance["Histogram"] = MACD(reliance["Close"])

# RSI
reliance["RSI"] = RSI(reliance["Close"])

# Stochastic Oscillator
reliance["%K"], reliance["%D"] = Stochastic(reliance["High"], reliance["Low"], reliance["Close"])

# Bollinger Bands
reliance["BB_Middle"], reliance["BB_Upper"], reliance["BB_Lower"] = Bollinger_Bands(reliance["Close"])

reliance.tail()

/tmp/ipython-input-3472994198.py:62: FutureWarning: YF.download() has changed argument auto_adjust default to True
  reliance = yf.download("RELIANCE.NS", period="2y")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,SMA_20,EMA_20,MACD,Signal,Histogram,RSI,%K,%D,BB_Middle,BB_Upper,BB_Lower
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,,,,,,,,,,,
Date,,,,,,,,,,,,,,,,
2025-12-11,1545.000000,1550.000000,1524.000000,1536.900024,4706197,1541.465009,Ticker RELIANCE.NS 1533.148294 dtype: float64,Ticker RELIANCE.NS 15.068814 dtype: float64,Ticker RELIANCE.NS 20.392491 dtype: float64,Ticker RELIANCE.NS -5.323678 dtype: float64,49.353806,43.014118,30.612274,1541.465009,1573.850386,1509.079631
2025-12-12,1556.500000,1559.800049,1546.099976,1550.800049,5106655,1543.345007,Ticker RELIANCE.NS 1535.372266 dtype: float64,Ticker RELIANCE.NS 15.235234 dtype: float64,Ticker RELIANCE.NS 19.36104 dtype: float64,Ticker RELIANCE.NS -4.125806 dtype: float64,58.266444,61.067472,44.793302,1543.345007,1574.559190,1512.130824
2025-12-15,1556.199951,1558.599976,1546.500000,1551.199951,5774543,1545.240002,Ticker RELIANCE.NS 1537.355855 dtype: float64,Ticker RELIANCE.NS 15.168064 dtype: float64,Ticker RELIANCE.NS 18.522444 dtype: float64,Ticker RELIANCE.NS -3.354381 dtype: float64,56.812546,60.596438,54.892676,1545.240002,1574.598850,1515.881154
2025-12-16,1542.300049,1551.699951,1538.800049,1545.000000,11731856,1546.385004,Ticker RELIANCE.NS 1537.82673 dtype: float64,Ticker RELIANCE.NS 13.833758 dtype: float64,Ticker RELIANCE.NS 17.584707 dtype: float64,Ticker RELIANCE.NS -3.750949 dtype: float64,36.832064,38.775581,53.479830,1546.385004,1573.174407,1519.595600
2025-12-17,1544.400024,1550.000000,1539.300049,1550.000000,7116136,1547.660004,Ticker RELIANCE.NS 1538.452758 dtype: float64,Ticker RELIANCE.NS 12.798231 dtype: float64,Ticker RELIANCE.NS 16.627412 dtype: float64,Ticker RELIANCE.NS -3.829181 dtype: float64,40.537839,42.072242,47.148087,1547.660004,1571.167880,1524.152128
